In [1]:
import pandas as pd
from datetime import datetime
from pytz import timezone
from functools import reduce
import os

## Preprocessing of Trade Balance Data

In [2]:
def form_dates(row, type_data):
    
    if type_data == 'trade' or type_data == 'money':
        months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
        years = row['Year'].split('-')
        month = row['Month']
        year = ""
        if month in ['January', 'February', 'March']:
            if int(years[1]) < 22:
                year = "20" + years[1]
            else: 
                year = "19" + years[1]
        else:
            year = years[0]
        month_i = months.index(month)+1
        if month_i < 10:
            return year+'-0'+str(month_i)+'-01'
        return year+'-'+str(month_i)+'-01'
    
    if type_data == 'exchange':
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        month, year = row['Year/Month'].split('-')
        month_i = months.index(month)+1
        if month_i < 10:
            return year+'-0'+str(month_i)+'-01'
        return year+'-'+str(month_i)+'-01'

In [3]:
path = "../data/India's_Foreign_Trade_-_US_Dollars.xlsx"

df = pd.read_excel(path, sheet_name='Report 1', header=5, index_col=0, skipfooter=2, usecols = "A:F")
df = df[df.Month != 'Annual']

df['Date'] = pd.to_datetime(df.apply (lambda row: form_dates(row, 'trade'), axis=1))
df = df.set_index('Date')

df = df[['Exports', 'Imports', 'Trade Balance']]

df.head()

,Exports,Imports,Trade Balance
Date,,,
2021-04-01,30733.212339,45751.188450,-15017.976111
2021-05-01,32290.353018,38554.444122,-6264.091104
2021-06-01,32497.690000,41871.890000,-9374.200000
2021-07-01,35427.930000,46401.290000,-10973.360000
2020-04-01,10159.834335,17083.408101,-6923.573766


In [4]:
# checking for null values:
print(df.isna().sum())

Exports          0
Imports          0
Trade Balance    0
dtype: int64


## Preprocessing of Exchange Rate Data

In [5]:
path = "../data/Exchange_Rate_of_the_Indian_Rupee_vis-a-vis_the_SDR,_US_Dollar,_Pound_Sterling_(Monthly_Average_and_End-Month_Rates).xlsx"

df1 = pd.read_excel(path, sheet_name='Report 1', header=6, skipfooter=1, usecols = "B:F")
df1 = df1[['Unnamed: 1', 'Average', 'Average.1']]
df1.columns = ['Year/Month', 'Exchange Rate (SDR)', 'Exchange Rate (USD)']

df1['Date'] = pd.to_datetime(df1.apply (lambda row: form_dates(row, 'exchange'), axis=1))
df1 = df1[['Exchange Rate (SDR)', 'Exchange Rate (USD)', 'Date']].set_index('Date')

df1.head()

,Exchange Rate (SDR),Exchange Rate (USD)
Date,,
2021-07-01,106.07421,74.52697
2021-06-01,105.58827,73.55826
2021-05-01,105.64294,73.26774
2021-04-01,106.39040,74.47221
2021-03-01,103.91876,72.79280


In [6]:
# checking for null values:
print(df1.isna().sum())

Exchange Rate (SDR)    0
Exchange Rate (USD)    0
dtype: int64


## Preprocessing of Money Stock Data

In [7]:
path = "../data/Components_of_Money_Stock.xlsx"
df2 = pd.read_excel(path, sheet_name='Report 1', header=5, skipfooter=2, usecols = "B:F")
df2 = df2.iloc[1:]

df2 = df2.fillna(method="ffill") # filling the NaN values in column 0 with the Year values preceding them

df2['Date'] = pd.to_datetime(df2.apply (lambda row: form_dates(row, 'money'), axis=1))
df2 = df2[['Currency in Circulation', 'Cash with Banks', 'Currency with the Public', 'Date']].set_index('Date')

df2.head()

,Currency in Circulation,Cash with Banks,Currency with the Public
Date,,,
2021-08-01,2.943861e+06,112940.1718,2.830920e+06
2021-07-01,2.946649e+06,107309.1785,2.839340e+06
2021-06-01,2.988519e+06,102503.9545,2.886015e+06
2021-05-01,2.964190e+06,102894.2701,2.861295e+06
2021-04-01,2.907160e+06,104322.7988,2.802837e+06


In [8]:
# checking for null values:
print(df2.isna().sum())

Currency in Circulation     0
Cash with Banks             0
Currency with the Public    0
dtype: int64


## Preprocessing of St Louis Fed Data

In [9]:
def process_excel(path, column, filetype):
    if filetype == 'not SA':
        df = pd.read_excel(path, header=10)
        df.columns = ['Date', column]
        df['Date'] = pd.to_datetime(df.Date)
        df = df.set_index('Date')
        df.head(5)
        return df.copy()
    
    if filetype == 'SA':
        df = pd.read_excel(path, sheet_name='SA_data', header=10)
        df.columns = ['Date', column]
        df['Date'] = pd.to_datetime(df.Date)
        df = df.set_index('Date')
        df.head(5)
        return df.copy()

In [10]:
path = "../data/fed_st_louis/seasonally adjusted"

df3 = process_excel(os.path.join(path, 'Consumer_Goods_Manufacture.xls'), 'Consumer Goods Manufacture', 'not SA')
df3.head()

,Consumer Goods Manufacture
Date,
1996-05-01,17.102181
1996-06-01,17.259676
1996-07-01,17.601184
1996-08-01,17.675383
1996-09-01,16.784550


In [11]:
df4 = process_excel(os.path.join(path, 'CPI_Seasonally_Adjusted.xlsx'), 'CPI', 'SA')
df4.head()

,CPI
Date,
1996-05-01,27.398863
1996-06-01,27.702649
1996-07-01,27.892160
1996-08-01,28.249249
1996-09-01,28.362334


In [12]:
df5 = process_excel(os.path.join(path, 'GDP.xls'), 'GDP', 'not SA')
df5.head(5)

,GDP
Date,
1996-05-01,101.000158
1996-06-01,100.917006
1996-07-01,100.834388
1996-08-01,100.752741
1996-09-01,100.672095


In [13]:
df6 = process_excel(os.path.join(path, 'GovernmentExpenditure.xls'), 'Government Expenditure', 'not SA')
df6.head()

,Government Expenditure
Date,
1996-04-01,3.486943e+11
1996-07-01,3.496083e+11
1996-10-01,3.605712e+11
1997-01-01,3.868893e+11
1997-04-01,3.808070e+11


In [14]:
df7 = process_excel(os.path.join(path, 'Gross_Fixed_Capital.xls'), 'Gross Fixed Capital', 'not SA')
df7.head()

,Gross Fixed Capital
Date,
1996-04-01,8.292596e+11
1996-07-01,8.671196e+11
1996-10-01,8.940429e+11
1997-01-01,9.379883e+11
1997-04-01,9.386363e+11


In [15]:
df8 = process_excel(os.path.join(path, 'Immediate_Rates_SA.xlsx'), 'Immediate Rates', 'SA')
df8.head()

,Immediate Rates
Date,
1996-05-01,12.132423
1996-06-01,12.191313
1996-07-01,12.026324
1996-08-01,11.967297
1996-09-01,11.947925


In [16]:
df9 = process_excel(os.path.join(path, 'Interest_Rates_SA.xlsx'), 'Interest Rates', 'SA')
df9.head()

,Interest Rates
Date,
1996-05-01,12.171535
1996-06-01,12.197480
1996-07-01,11.982626
1996-08-01,11.962707
1996-09-01,11.955910


In [17]:
df10 = process_excel(os.path.join(path, 'Share_Prices_SA.xlsx'), 'Share Prices', 'SA')
df10.head(5)

,Share Prices
Date,
1996-05-01,56.942086
1996-06-01,55.909167
1996-07-01,9.847059
1996-08-01,1.132843
1996-09-01,-0.106304


In [18]:
df11 = process_excel(os.path.join(path, 'Wholesale_Industry_Prices_SA.xlsx'), 'Wholesale Industry Prices', 'SA')
df11.head(5)

,Wholesale Industry Prices
Date,
1996-05-01,38.828191
1996-06-01,38.887180
1996-07-01,39.379901
1996-08-01,39.582129
1996-09-01,39.665158


In [19]:
# checking for null values:
print(df3.isna().sum(), end='\n\n')
print(df4.isna().sum(), end='\n\n')
print(df5.isna().sum(), end='\n\n')
print(df6.isna().sum(), end='\n\n')
print(df7.isna().sum(), end='\n\n')
print(df8.isna().sum(), end='\n\n')
print(df9.isna().sum(), end='\n\n')
print(df10.isna().sum(), end='\n\n')
print(df11.isna().sum(), end='\n\n')

Consumer Goods Manufacture    0
dtype: int64

CPI    0
dtype: int64

GDP    0
dtype: int64

Government Expenditure    0
dtype: int64

Gross Fixed Capital    0
dtype: int64

Immediate Rates    0
dtype: int64

Interest Rates    0
dtype: int64

Share Prices    0
dtype: int64

Wholesale Industry Prices    0
dtype: int64



## Merging the Dataframes along their common index

In [20]:
dataframes = [df, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]

# reference code: https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
data = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='inner'), dataframes)

data = data.sort_index()

data.head()

,Exports,Imports,Trade Balance,Currency in Circulation,Cash with Banks,Currency with the Public,Consumer Goods Manufacture,CPI,GDP,Government Expenditure,Gross Fixed Capital,Immediate Rates,Interest Rates,Share Prices,Wholesale Industry Prices
Date,,,,,,,,,,,,,,,
1996-07-01,2689.0,2937.0,-248.0,129229.0,4250.0,124979.0,17.601184,27.892160,100.834388,3.496083e+11,8.671196e+11,12.026324,11.982626,9.847059,39.379901
1996-10-01,2665.0,3228.0,-563.0,127908.0,4276.0,123632.0,17.368679,28.400053,100.591658,3.605712e+11,8.940429e+11,12.104385,12.112597,3.705483,39.557911
1997-01-01,2941.0,3882.0,-941.0,132945.0,4357.0,128588.0,17.364274,29.060325,100.344831,3.868893e+11,9.379883e+11,11.885136,11.875432,12.452448,40.147988
1997-04-01,2635.0,3180.0,-545.0,143581.0,4411.0,139170.0,17.743765,29.603703,100.109642,3.808070e+11,9.386363e+11,11.042559,11.073390,-2.033017,40.813951
1997-07-01,2988.0,3553.0,-566.0,147880.0,4787.0,143093.0,18.515714,29.455437,99.950991,3.875761e+11,9.518487e+11,10.021937,9.985522,15.788213,40.806259


In [21]:
# Merged data details

print(len(data)) # number of rows
print()
# printing out the date ranges of the data
print(data.first_valid_index())
print(data.last_valid_index())

100

1996-07-01 00:00:00
2021-04-01 00:00:00


## Save Cleaned and Merged Data

In [22]:
data.to_csv('../data/merged_data.csv')